In [1]:
from celery import Celery
import os
from dotenv import load_dotenv
from ollama import Client
import httpx
from langchain_community.chat_models.ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

In [2]:
# markdown 파일 생성
def save_md_to_file(markdown_str : str, directory : str, file_name : str) :
    # 디렉토리가 존재하지 않으면 생성
    if not os.path.exists(directory):
        os.makedirs(directory)
        
     # 파일 경로 생성
    file_path = os.path.join(directory, file_name)
    
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(markdown_str)
    
    # 파일 객체 정보 변환
    file_info = {
        'file_path' : file_path,
        'file_name' : os.path.getsize(file_path)
    }
    
    return file_info

In [3]:
# 각 파일당 하나씩 프롬프트 생성하기
user_prompt = '''
You are an experienced technical writer skilled in documenting and formatting code reviews. 
Your task is to take the results from a code review and format them into a markdown file. 
The markdown file must be written in English and should include the following sections: Analysis Summary, Key Features, Pre-condition Check, Runtime Error Check, Optimization, Security Issue, and Evaluation. 
Each section should be formatted appropriately:

- **Analysis Summary:** Summarize the code review findings in one or two lines.
- **Key Features:** Analyze and describe the key features of the added or modified files.
- **Precondition checks:** Checks that a function or method has the necessary variable states or ranges of values to function correctly.
- **Runtime error checking:** Examines code for possible runtime errors and identifies other potential risks.
- **Optimization:** Scan your code and recommend optimized code. When recommending code, be sure to include the full source of the file. Please write your code using code blocks to conform to the markdown format - this is a must. 
- **Security issues:** Scans your code to see if it uses modules with serious security flaws or contains security vulnerabilities.
- **Evaluation:** Comprehensively evaluates your work. Consider the quality, functionality, and maintainability of the code.
            
Ensure the markdown document is clear, well-structured, and easy to read.
```python
# main.py
import requests
from bs4 import BeautifulSoup
from langchain.agents import initialize_agent, AgentType
import json
import random
import re
from fastapi.requests import Request
from fastapi import FastAPI, HTTPException, Query, Header, Depends
from dotenv import load_dotenv
import os
from langchain.schema import StrOutputParser
import time
from urllib.parse import urlparse
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage

```
'''

In [4]:
# llm 생성
llama_llm = ChatOllama(           
    base_url = "http://localhost:11434",
    model="llama3.1",
    temperature=0,
)     
mistral_llm = ChatOllama(
    base_url = "http://localhost:11434",
    model="mistral",
    temperature=0,
)     


In [5]:
# prompt 생성
code_review_messages = ChatPromptTemplate.from_messages([
    ("system", """
    You're a very good software analyst. From now on, users will show you the entire committed source code. Take a look at it, analyze it, and tell us what you find. 
    After looking at the source code, present an optimized version of the code, including performance improvements. 

    As you analyze the source code, keep the following topics in mind as you do so
    - Analysis summary: Summarize your code review findings in one or two lines.
    - Key features: Analyze and describe the key features of each file.
    - Prerequisite checks: Verify that the function or method has the necessary variable states or value ranges to function correctly.
    - Runtime error checking: Inspect your code for possible runtime errors and identify other potential risks.
    - Optimization: Scans code patches for optimization points and recommends optimized code if it appears to be degrading performance. 
    - Security issues: Scan your code to see if it uses modules with serious security flaws or contains security vulnerabilities.
    - Evaluation: Evaluate your work comprehensively. Considers the quality, functionality, and maintainability of the code.

    Return a response in markdown so that your analysis is easy to parse.
    The topics above are the same as the subheadings in your final analysis. In particular, be sure to write the entire code in the form of code blocks in Markdown for optimizations.
    The Markdown documentation must be written in Korean.
    Do not write any additional text other than the response values in Markdown format.
    """),
    ("human", "{prompt}")
])
translate_messages = ChatPromptTemplate.from_messages([
    ("system", """
    You are an excellent translator. Please translate the markdown written by the user into Korean. 
    At this time, for the content in the code block, only the annotation should be translated, and the code should be left as it is.
    """),
    ("human", "{review_result}")
])

In [6]:
# chain 생성
# 1. 코드 분석 실행. 어떤 코드고, 리팩토링하면 좋은 것 까지.
review_chain = code_review_messages | llama_llm

In [7]:
# 2. 해당 코드 분석을 포멧에 맞게 변환.
translate_chain = translate_messages | mistral_llm

In [8]:
# 체인 실행
final_chain = {"review_result" : review_chain} | translate_chain

In [9]:
# 데이터 리턴
result = final_chain.invoke({
    "prompt" : user_prompt
})

In [10]:
# 파일생성
file_dir = "./"
file_name = "example.md"
file_info = save_md_to_file(result.content, file_dir, file_name)